Normaliza nombres de columnas: minúsculas, sin espacios ni acentos.

In [1]:
import os
import pandas as pd

ruta_excel = "../data/raw/Sales report completa.xlsx"
df = pd.read_excel(ruta_excel)
print(df.head(0))


Empty DataFrame
Columns: [Número Venta, Mes Salida, Dia Salida, Año Salida, Mes Entrega, Dia Entrega, Año Entrega, Método Envio, Número Cliente, Nombre Cliente, Segmento, Ciudad, Estado, País, ID Producto, Ventas, Cantidad, Descuento, Utilidad, Costo Envío, Prioridad Envio]
Index: []

[0 rows x 21 columns]


In [2]:


def estandarizar_columnas(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
         .str.replace("-", "_")
        .str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
    )
    return df 
# Aplicar la función al dataframe que ya cargaste
df = estandarizar_columnas(df)

# Mostrar las nuevas columnas estandarizadas
print(df.columns)

Index(['numero_venta', 'mes_salida', 'dia_salida', 'ano_salida', 'mes_entrega',
       'dia_entrega', 'ano_entrega', 'metodo_envio', 'numero_cliente',
       'nombre_cliente', 'segmento', 'ciudad', 'estado', 'pais', 'id_producto',
       'ventas', 'cantidad', 'descuento', 'utilidad', 'costo_envio',
       'prioridad_envio'],
      dtype='object')


Estandariza las tres primeras hojas del archivo Excel (Ventas Supermercado, Regiones, y Productos)

In [3]:
# Cargar cada hoja por separado
ventas_df = pd.read_excel(ruta_excel, sheet_name=0)          # Primera hoja (Ventas Supermercado)
regiones_df = pd.read_excel(ruta_excel, sheet_name="Regiones")
productos_df = pd.read_excel(ruta_excel, sheet_name="Productos")

# Estandarizar las columnas de cada uno
ventas_df = estandarizar_columnas(ventas_df)
regiones_df = estandarizar_columnas(regiones_df)
productos_df = estandarizar_columnas(productos_df)

# Mostrar los nombres de columnas como verificación
print("Columnas en hoja Ventas:", ventas_df.columns.tolist())
print("Columnas en hoja Regiones:", regiones_df.columns.tolist())
print("Columnas en hoja Productos:", productos_df.columns.tolist())


Columnas en hoja Ventas: ['numero_venta', 'mes_salida', 'dia_salida', 'ano_salida', 'mes_entrega', 'dia_entrega', 'ano_entrega', 'metodo_envio', 'numero_cliente', 'nombre_cliente', 'segmento', 'ciudad', 'estado', 'pais', 'id_producto', 'ventas', 'cantidad', 'descuento', 'utilidad', 'costo_envio', 'prioridad_envio']
Columnas en hoja Regiones: ['pais', 'mercado', 'region']
Columnas en hoja Productos: ['id_producto', 'categoria', 'sub_categoria', 'nombre_producto']


Elimina los espacios en blanco innecesarios de la columna "Método de envío" Ejemplo: " Standard  	Class " por "Standard Class"

In [4]:
def limpiar_espacios_metodo_envio(df: pd.DataFrame) -> pd.DataFrame:
    if 'metodo_envio' in df.columns:
        df['metodo_envio'] = df['metodo_envio'].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)
    else:
        print("La columna 'metodo_envio' no existe en el DataFrame.")
    return df

# Aplicar la función
df = limpiar_espacios_metodo_envio(df)

# Ver los valores únicos de la columna después de limpiar
print(df['metodo_envio'].unique())


['Standard Class' 'Second Class' 'First Class' 'Same Day']


Unificar Mes/Dia/Año de las fecha_envio y fecha_entrega

In [5]:
def convertir_fechas(df: pd.DataFrame) -> pd.DataFrame:
    # Convertir las columnas de año, mes y día a números enteros (permitiendo nulos)
    for col in ['ano_salida', 'mes_salida', 'dia_salida', 'ano_entrega', 'mes_entrega', 'dia_entrega']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')  # 'Int64' permite NaN

    # Crear columna de fecha_salida si existen las tres columnas necesarias
    if {'ano_salida', 'mes_salida', 'dia_salida'}.issubset(df.columns):
        df['fecha_salida'] = pd.to_datetime(
            df[['ano_salida', 'mes_salida', 'dia_salida']].rename(
                columns={
                    'ano_salida': 'year',
                    'mes_salida': 'month',
                    'dia_salida': 'day'
                }
            ),
            errors='coerce'
        )
        
    # Crear columna de fecha_entrega si existen las tres columnas necesarias
    if {'ano_entrega', 'mes_entrega', 'dia_entrega'}.issubset(df.columns):
        df['fecha_entrega'] = pd.to_datetime(
            df[['ano_entrega', 'mes_entrega', 'dia_entrega']].rename(
                columns={
                    'ano_entrega': 'year',
                    'mes_entrega': 'month',
                    'dia_entrega': 'day'
                }
            ),
            errors='coerce'
        )

    return df

# Aplicar la función al DataFrame
df = convertir_fechas(df)

# Verificar las nuevas columnas de fecha
print(df[['fecha_salida', 'fecha_entrega']].head())
print(df.columns)


  fecha_salida fecha_entrega
0   2011-01-01    2011-01-06
1   2011-01-01    2011-01-08
2   2011-01-01    2011-01-05
3   2011-01-01    2011-01-05
4   2011-01-01    2011-01-08
Index(['numero_venta', 'mes_salida', 'dia_salida', 'ano_salida', 'mes_entrega',
       'dia_entrega', 'ano_entrega', 'metodo_envio', 'numero_cliente',
       'nombre_cliente', 'segmento', 'ciudad', 'estado', 'pais', 'id_producto',
       'ventas', 'cantidad', 'descuento', 'utilidad', 'costo_envio',
       'prioridad_envio', 'fecha_salida', 'fecha_entrega'],
      dtype='object')


Agrega una columna con los dias_transcurridos" entre la Fecha Salida y Fecha 
Entrega. 

In [6]:
def calcular_duracion_envio(df: pd.DataFrame) -> pd.DataFrame:
    if {'fecha_salida', 'fecha_entrega'}.issubset(df.columns):
        df['duracion_envio'] = (df['fecha_entrega'] - df['fecha_salida']).dt.days
    else:
        print("Las columnas 'fecha_salida' y/o 'fecha_entrega' no existen en el DataFrame.")
    return df

# Aplicar la función al DataFrame
df = calcular_duracion_envio(df)

# Verificar resultado
print(df[['fecha_salida', 'fecha_entrega', 'duracion_envio']].head())


  fecha_salida fecha_entrega  duracion_envio
0   2011-01-01    2011-01-06               5
1   2011-01-01    2011-01-08               7
2   2011-01-01    2011-01-05               4
3   2011-01-01    2011-01-05               4
4   2011-01-01    2011-01-08               7


Agrega una columna con  id_venta unificando y pasando a mayusc: 2 iniciales de 
"País", "Año Salida" y "Número Venta" (ejemplo: Al-2011-2040). 


In [7]:
def crear_id_venta(df: pd.DataFrame) -> pd.DataFrame:
    if {'pais', 'ano_salida', 'numero_venta'}.issubset(df.columns):
        # Asegurar que los datos sean cadenas y manejar valores faltantes
        df['id_venta'] = (
            df['pais'].astype(str).str.strip().str[:2]      # Primeras 2 letras del país
            + '-' +
            df['ano_salida'].astype(str).str.strip()        # Año de salida
            + '-' +
            df['numero_venta'].astype(str).str.strip()      # Número de venta
        ).str.upper()  # Convertir todo a mayúsculas
    else:
        print("Faltan una o más columnas necesarias: 'pais', 'ano_salida', 'numero_venta'")
    return df

# Aplicar la función
df = crear_id_venta(df)

# Verificar las primeras filas
print(df[['pais', 'ano_salida', 'numero_venta', 'id_venta']].head(2))



        pais  ano_salida  numero_venta       id_venta
0    Algeria        2011          2040   AL-2011-2040
1  Australia        2011         47883  AU-2011-47883


Agrega una columna con id_cliente unificando: Inicial Nombre, Inicial Apellido y 
Número Cliente (ejemplo: TB-11280).  

In [8]:
def crear_id_cliente_desde_nombre_completo(df: pd.DataFrame) -> pd.DataFrame:
    if {'nombre_cliente', 'numero_cliente'}.issubset(df.columns):
        # Separar nombre y apellido suponiendo que hay al menos dos palabras
        partes = df['nombre_cliente'].astype(str).str.strip().str.split(expand=True)

        # Tomar primera palabra como nombre y segunda como apellido (si existen)
        df['inicial_nombre'] = partes[0].str[0]
        df['inicial_apellido'] = partes[1].str[0] if partes.shape[1] > 1 else ''

        # Crear el id_cliente combinando iniciales + número_cliente
        df['id_cliente'] = (
            df['inicial_nombre'].fillna('') +
            df['inicial_apellido'].fillna('') +
            '-' +
            df['numero_cliente'].astype(str).str.strip()
        ).str.upper()

        # (Opcional) Eliminar las columnas auxiliares si no se quieren mostrar
        df.drop(columns=['inicial_nombre', 'inicial_apellido'], inplace=True)

    else:
        print("Faltan columnas necesarias: 'nombre_cliente' y/o 'numero_cliente'")
    return df

# Aplicar la función
df = crear_id_cliente_desde_nombre_completo(df)

# Verificar resultado
print(df[['nombre_cliente', 'numero_cliente', 'id_cliente']].head(2))



    nombre_cliente  numero_cliente id_cliente
0  Toby Braunhardt           11280   TB-11280
1      Joseph Holt           15985   JH-15985


Agrega las columnas mercado y región obteniéndose de la columna País y la Hoja 
Regiones dentro del mismo archivo excel.

In [12]:
def agregar_mercado_region(df: pd.DataFrame, regiones_df: pd.DataFrame) -> pd.DataFrame:
    try:
        # Asegurarse de que la columna 'pais' esté limpia en ambos DataFrames
        df['pais'] = df['pais'].astype(str).str.strip()
        regiones_df['pais'] = regiones_df['pais'].astype(str).str.strip()

        # Unir los DataFrames por la columna 'pais'
        df = df.merge(regiones_df[['pais', 'mercado', 'region']], on='pais', how='left')

    except Exception as e:
        print(f"Error al agregar mercado y región: {e}")

    return df


# Verificar las nuevas columnas
print(df.head(2))



   numero_venta  mes_salida  dia_salida  ano_salida  mes_entrega  dia_entrega  \
0          2040           1           1        2011            1            6   
1         47883           1           1        2011            1            8   

   ano_entrega    metodo_envio  numero_cliente   nombre_cliente  ... cantidad  \
0         2011  Standard Class           11280  Toby Braunhardt  ...        2   
1         2011  Standard Class           15985      Joseph Holt  ...        3   

  descuento utilidad costo_envio prioridad_envio  fecha_salida  fecha_entrega  \
0       0.0  106.140       35.46          Medium    2011-01-01     2011-01-06   
1       0.1   36.036        9.72          Medium    2011-01-01     2011-01-08   

   duracion_envio       id_venta  id_cliente  
0               5   AL-2011-2040    TB-11280  
1               7  AU-2011-47883    JH-15985  

[2 rows x 26 columns]


Agrega las columnas categoría, sub_categoría y nombre_producto obteniéndolas de la 
columna ID-Producto y la Hoja Productos.

In [ ]:
def agregar_mercado_region(df: pd.DataFrame, regiones_df: pd.DataFrame) -> pd.DataFrame:
    try:
        # Limpiar la columna 'pais' en ambos DataFrames
        df['pais'] = df['pais'].astype(str).str.strip()
        regiones_df['pais'] = regiones_df['pais'].astype(str).str.strip()

        # Unir por 'pais'
        df = df.merge(regiones_df[['pais', 'mercado', 'region']], on='pais', how='left')

    except Exception as e:
        print(f"Error al agregar mercado y región: {e}")

    return df


# Verificar resultado
print(df[['id_producto', 'categoria', 'sub_categoria', 'nombre_producto']].drop_duplicates().head(2))
print(df.columns)


        id_producto        categoria sub_categoria           nombre_producto
0  OFF-TEN-10000025  Office Supplies       Storage       Tenex Lockers, Blue
1   OFF-SU-10000618  Office Supplies      Supplies  Acme Trimmer, High Speed
Index(['numero_venta', 'mes_salida', 'dia_salida', 'ano_salida', 'mes_entrega',
       'dia_entrega', 'ano_entrega', 'metodo_envio', 'numero_cliente',
       'nombre_cliente', 'segmento', 'ciudad', 'estado', 'pais', 'id_producto',
       'ventas', 'cantidad', 'descuento', 'utilidad', 'costo_envio',
       'prioridad_envio', 'fecha_salida', 'fecha_entrega', 'duracion_envio',
       'id_venta', 'id_cliente', 'mercado', 'region', 'categoria_x',
       'sub_categoria_x', 'nombre_producto_x', 'categoria_y',
       'sub_categoria_y', 'nombre_producto_y', 'categoria', 'sub_categoria',
       'nombre_producto'],
      dtype='object')


In [ ]:
def agregar_resultado_venta(df: pd.DataFrame) -> pd.DataFrame:
    try:
        # Verificar que exista la columna 'utilidad'
        if 'utilidad' in df.columns:
            df['resultado_venta'] = df['utilidad'].apply(
                lambda x: 'Ganancia' if x > 0 else ('Pérdida' if x < 0 else 'Neutro')
            )
        else:
            print("Columna 'utilidad' no encontrada en el DataFrame.")

    except Exception as e:
        print(f"Error al agregar columna 'resultado_venta': {e}")

    return df

ventas_df = agregar_resultado_venta(ventas_df)

# Verificamos el resultado
print(ventas_df[['utilidad', 'resultado_venta']].head())


   utilidad resultado_venta
0   106.140        Ganancia
1    36.036        Ganancia
2    29.640        Ganancia
3   -26.055         Pérdida
4    37.770        Ganancia


In [ ]:
print(df.columns)
print(df.head())

Index(['numero_venta', 'mes_salida', 'dia_salida', 'ano_salida', 'mes_entrega',
       'dia_entrega', 'ano_entrega', 'metodo_envio', 'numero_cliente',
       'nombre_cliente', 'segmento', 'ciudad', 'estado', 'pais', 'id_producto',
       'ventas', 'cantidad', 'descuento', 'utilidad', 'costo_envio',
       'prioridad_envio', 'fecha_salida', 'fecha_entrega', 'duracion_envio',
       'id_venta', 'id_cliente', 'mercado', 'region', 'categoria_x',
       'sub_categoria_x', 'nombre_producto_x', 'categoria_y',
       'sub_categoria_y', 'nombre_producto_y', 'categoria', 'sub_categoria',
       'nombre_producto'],
      dtype='object')
   numero_venta  mes_salida  dia_salida  ano_salida  mes_entrega  dia_entrega  \
0          2040           1           1        2011            1            6   
1         47883           1           1        2011            1            8   
2          1220           1           1        2011            1            5   
3       3647632           1           1   

Salida

In [10]:
# Crear carpeta si no existe
import os

# Definir ruta de salida
output_folder = "../data/processed"
os.makedirs(output_folder, exist_ok=True)

# Rutas completas para guardar
ruta_excel_final = os.path.join(output_folder, "sales_report_final.xlsx")
ruta_csv_final = os.path.join(output_folder, "sales_report_final.csv")

# Guardar en formato Excel
df.to_excel(ruta_excel_final, index=False)

# Guardar en formato CSV
df.to_csv(ruta_csv_final, index=False, encoding='utf-8')

print("Archivos exportados correctamente:")
print("✔ Excel:", ruta_excel_final)
print("✔ CSV:", ruta_csv_final)


Archivos exportados correctamente:
✔ Excel: ../data/processed\sales_report_final.xlsx
✔ CSV: ../data/processed\sales_report_final.csv
